# xgboost 학습
만들어진 Doc2vec모델을 xgboost로 이용하여 학습시킨다.

#  1. 테스트 및 학습을 위한 특징 가져오기

In [31]:
import csv

In [6]:
#테스트와 학습을 위해 각각의 문자열을 가져온다.
with open('./software_feature/behavior_api_order.csv', 'r', encoding='utf-8') as f:
    lines = csv.reader(f)
    software_sentences = [line for line in lines]
with open('./malware_feature/behavior_api_order.csv', 'r', encoding='utf-8') as f:
    lines = csv.reader(f)
    malware_sentences = [line for line in lines]

In [7]:
software_sentences[5]

['GetSystemTimeAsFileTime',
 'LdrLoadDll',
 'LdrLoadDll',
 'LdrGetProcedureAddress',
 'LdrLoadDll',
 'LdrGetProcedureAddress',
 'LdrGetProcedureAddress',
 'LdrLoadDll',
 'LdrLoadDll',
 'LdrGetProcedureAddress',
 'LdrLoadDll',
 'LdrGetProcedureAddress',
 'LdrGetProcedureAddress',
 'LdrGetProcedureAddress',
 'LdrLoadDll',
 'LdrGetProcedureAddress',
 'LdrGetProcedureAddress',
 'NtAllocateVirtualMemory',
 'LdrGetDllHandle',
 'LdrGetDllHandle',
 'LdrGetDllHandle',
 'LdrGetProcedureAddress',
 'LdrGetProcedureAddress',
 'LdrGetProcedureAddress',
 'SetUnhandledExceptionFilter',
 'CoInitializeEx',
 'NtAllocateVirtualMemory',
 'NtClose',
 'RegOpenKeyExW',
 'RegQueryValueExW',
 'RegOpenKeyExW',
 'RegQueryValueExW',
 'RegCloseKey',
 'RegCloseKey',
 'NtClose',
 'RegOpenKeyExW',
 'NtClose',
 'RegOpenKeyExW',
 'RegOpenKeyExW',
 'FindResourceExW',
 'LoadResource',
 'SizeofResource',
 'SHGetFolderPathW',
 'CreateDirectoryW',
 'CreateDirectoryW',
 'FindResourceExW',
 'LoadResource',
 'SizeofResource',
 

#  2. Doc2vec 모델을 사용하여 벡터화 시키기

In [8]:
from gensim.models.doc2vec import Doc2Vec

C:\Users\0505z\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
#학습에 사용할 doc2vec모델 load 시킴
doc2vec_model_name = "Doc2vec_model_vector30_window15_dm0"
model = Doc2Vec.load(doc2vec_model_name)

In [10]:
#위에서 가져온 문자열들을 doc2vec model에 넣고 벡터로 변환
#오래걸림.
software_vector = [model.infer_vector(sentence,alpha=0.025,min_alpha=0.025, epochs=30)
                 for sentence in software_sentences]
malware_vector = [model.infer_vector(sentence,alpha=0.025,min_alpha=0.025, epochs=30)
                 for sentence in malware_sentences]

In [11]:
software_vector[0]

array([ 0.24579558,  0.6292196 ,  0.49041584, -0.907344  , -0.1858422 ,
       -0.606968  , -0.13083784,  0.36080185,  0.46417585,  0.9371711 ,
        0.84641993, -0.0170335 , -0.2354685 ,  0.5720206 , -0.29250947,
        0.21936749, -0.86938363,  0.04338413, -0.1315153 ,  0.02578575,
       -0.4416653 ,  0.20575936, -0.02486834,  0.6473304 , -0.18889195,
        0.5294036 ,  0.0762881 , -0.434419  ,  0.09805749, -0.15409985,
       -0.07039478, -0.34660837, -1.163556  ,  0.2916624 , -0.05340414,
        0.20066278,  0.27178195, -0.4133039 ,  0.20415413, -0.10779564,
       -0.5288024 , -0.19333586,  0.02672192, -0.10108104,  0.16519757,
        0.4271366 ,  0.02308673,  0.44346496,  0.11172476, -0.33004352],
      dtype=float32)

#  3. xgboost 학습을 위한 전처리

In [12]:
import numpy as np

In [13]:
# xgboost의 학습을 위한 전처리 과정으로 vecotr list을 numpy의 array로 변환
software_arrays = np.array(software_vector)
malware_arrays = np.array(malware_vector)

In [14]:
type(software_arrays)

numpy.ndarray

In [15]:
type(software_vector)

list

In [16]:
# software 개수만큼 0으로 초기화된 label 생성
software_labels = np.zeros(len(software_vector))

# malware 개수만큼 1으로 초기화된 label 생성
malware_labels = np.ones(len(malware_vector))

In [17]:
software_labels#xgboost 에서 소프트웨어 라벨로 쓰임.

array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
malware_labels#xgboost 에서 악성코드 라벨로 쓰임.

array([1., 1., 1., ..., 1., 1., 1.])

In [19]:
#데이터를 하나로 합치기. 
arrays = np.vstack((software_arrays,malware_arrays))
labels = np.hstack((software_labels,malware_labels))

In [20]:
arrays

array([[ 0.24579558,  0.6292196 ,  0.49041584, ...,  0.44346496,
         0.11172476, -0.33004352],
       [-0.9174151 , -0.53251356, -0.5084604 , ..., -0.34203508,
         0.5504905 ,  1.4696544 ],
       [ 0.05081558,  0.32919607,  0.643398  , ...,  0.7511018 ,
         0.05108057,  0.37183863],
       ...,
       [ 0.85057485,  0.09666751,  0.89462215, ...,  0.44601947,
        -0.36087343, -0.29069513],
       [ 0.25964344, -0.29441205, -0.01333169, ...,  0.05367242,
         0.07317493, -1.3837619 ],
       [ 0.40780887,  0.04864832, -0.3554654 , ...,  0.42935917,
        -0.65671545, -1.0234853 ]], dtype=float32)

In [21]:
labels

array([0., 0., 0., ..., 1., 1., 1.])

# 4. xgboost 학습

In [22]:
import xgboost as xgb
from sklearn.model_selection import KFold
import collections
from bayes_opt import BayesianOptimization

참고 사이트: https://communedeart.com/2017/06/25/xgboost-사용하기/

1.General Parameters
    - booster[default: gbtree]: 부스터 구조를 정의 옵션: gbtree, gblinear, dart
    - nthread: 쓰레드 개수 default = 최대한
    - num_feature: 차원의 숫자. default = 최대한
    - silent[default = 0]: True 로 할 경우, running message가 출력이 안됨. False 추천

2.Booster Parameters(12 가지정도가 존재, min_child_weight, max_depth, gamma는 튜닝 추천)
    - eta: learning rate(학습률). 트리에 가지가 많을 수록 과적합(overfitting) 되기 쉬움
           매 부스팅 스탭 마다 가중치(weight)를 주어 과적합이 일어나지 않도록 해야함.
    - min_child_weight[default = 1]: overfitting을 조절. 값이 높아지면 unber-fitting 되는 경우가 있음.
    - gamma[default = 0]: 정보 획득(information Gain) -r / IG가 커지면 트리 깊이가 줄어 보수적인 모델이 됨.
    - max_depth[default = 6]: 한 트리의 maximum depth. 숫자를 키울수록 모델의 복잡도가 커짐. 과적합(overfitting) 되기 쉬움
                              leaf node 개수는 최대 2^6 64개/ Typical value 3~10
    - subsample: 개별트리에 대해 칼럼들이 랜덤하게 샘플되는 비율 보통 안씀
    - lambda(L2 reg-form): L2 Regularization Form의 가중치(weights) 숫자가 클수록 보수적인 모델
    - alpha(L1 reg-form): L1 Regularization Form 가중치(weights). 숫자가 클수록 보수적인 모델

3.Learning Task Parameters
    - objective: reg:linear(linear-regression),
                 binary:logistic(binary-logistic classification) 이항분류를 위한 로지스틱 회귀, 클래스가 아닌 예측 확률 반환
                 count:poisson(count data poison regression)
                 multi:softmax 다분류를 위한 softmax objective 예측한 클래스를 반환. num_class 파라미터를 써야함. 클래스 수
                 mulit:softprob 위와 동일, 확률의 예측값을 반환
    - eval_metric: 모델의 평가 함수를 조정하는 함수.
                 rmse(root mean square error),
                 logloss(log-likelihood),
                 map(mean average precision)
                 rma(mean absolute error)
                 logloss(negative log-likelihood)
                 error(binary classification error rate)
                 merror(multiclass classification error rate)
                 mlogloss(multiclass logloss)
                 auc(area under the curve)
    - seed[default = 0]:  시드 값 고정.

4.Command Line Parameters
    - num_rounds: boosting round를 결정. 랜덤하게 생성되는 모델이니 만큼 큰게 좋다고함.
                  많이 커봐야 오래기다리기만함. epoch 옵션과 동일

파라메터를 제대로 이해하고 있어야 모델 구축을 하는데 시행착오가 적어진다고함.
ex) 이진 분류(binary-classification) 문제에 linear-regression을 적용해놓고,
    L1, L2 파라메터를 조정하고 있다면, 큰 시간을 낭비

블로그 글쓴이가 밝힌 팁(파라메터 우선 순위)
1. booster(부스터 모양)
2. eval_metric(평가 함수)/objective(목적함수)
3. eta(learning rate)
4. L1 form
5. L2 form

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
깃허브 글쓴이가 밝힌 팁.
1. Booster Parameters에서 min_child_weight, max_depth, gamma는 반드시 튜닝 추천

소스코드 출저는 여기
http://codingwiththomas.blogspot.com/2016/10/xgboost-bayesian-hyperparameter-tuning.html

kFoldValidation 함수는 수정 함. 

In [50]:
#kFold로 학습시킨후 평균을 내어 반환 하는 함수.
def kFoldValidation(train, features, xgbParams, numRounds, nFolds, target='loss'):#가장 마지막으로 호출됨.
    kf = KFold(n_splits = nFolds, shuffle = True)
    fold_score=[]
    for train_index, test_index in kf.split(train):
        # split train/validation
        X_train, X_valid = train[train_index], train[test_index]
        y_train, y_valid = features[train_index], features[test_index]
        dtrain = xgb.DMatrix(X_train, y_train)
        dvalid = xgb.DMatrix(X_valid, y_valid)
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        gbm = xgb.train(xgbParams, dtrain, numRounds, evals = watchlist,early_stopping_rounds = 50,verbose_eval=False)#verbose 옵션으로 나타나지 않게함
        score = gbm.best_score
        fold_score.append(score)
    return np.mean(fold_score)
def xgbCv(train, features, numRounds, eta, gamma, maxDepth, minChildWeight, subsample, colSample):
    # prepare xgb parameters
    params = {
        "objective": "reg:linear",
        "booster" : "gbtree",
        "eval_metric": "mae",
        "tree_method": 'auto',
        "silent": 1,
        "eta": eta,
        "max_depth": int(maxDepth),
        "min_child_weight" : minChildWeight,
        "subsample": subsample,
        "colsample_bytree": colSample,
        "gamma": gamma,
        "scale_pos_weight": 0.48#데이터 셋 비율  0/1 소프트웨어 / 악성코드
        }#적용할 파라미터들
    #순서대로 train 학습시킬 데이터, features 특징, 기준이 될 xgb 파라미터, numRounds 반복횟수, nFolds =
    cvScore = kFoldValidation(train, features, params, int(numRounds), nFolds = 5)
    print('CV score: {:.6f}'.format(cvScore))
    return -1.0 * cvScore   # invert the cv score to let bayopt maximize
def bayesOpt(train, features):#가장먼저 호출됨.
    #train = train_data#테스트용 지울것
    #features = train_labels#테스트용 지울것
    ranges = {
        'numRounds': (1000, 2000),
        'eta': (0.03, 0.1),
        'gamma': (0, 10),
        'maxDepth': (4, 10),
        'minChildWeight': (0, 10),
        'subsample': (0, 1),
        'colSample': (0, 1),
        }#학습에 따라 변경될 값들.
    # proxy through a lambda to be able to pass train and features
    optFunc = lambda numRounds, eta, gamma, maxDepth, minChildWeight, subsample, colSample: xgbCv(train, features, numRounds, eta, gamma, maxDepth, minChildWeight, subsample, colSample)
    bo = BayesianOptimization(optFunc, ranges)
    bo.maximize(init_points = 50, n_iter = 5, kappa = 2, acq = "ei", xi = 0.0)
    bestMAE = round((-1.0 * bo.res['max']['max_val']), 6)
    print("\n Best MAE found: %f" % bestMAE)
    print("\n Parameters: %s" % bo.res['max']['max_params'])#가장 최고의 파라미터 추출 나중에 리턴 받으면 될듯
    p = bo.res['max']['max_params']#가장 좋은 파라미터를 찾아서 p에 넣음.
    max_params = {
        'eta': float(p['eta']),
        'max_depth': int(p['maxDepth']),
        'subsample': max(min(p['subsample'], 1), 0),
        'objective': 'reg:linear',
        'silent': 1,
        'min_child_weight': int(p['minChildWeight']),
        'gamma': max(p['gamma'], 0),
        "objective": "reg:linear",
        "booster" : "gbtree",
        "eval_metric": "mae",
        "tree_method": 'auto',
        "silent": 1,
        "scale_pos_weight": 0.48
        }#가장 좋은 파라미터를 저장.

    with open(doc2vec_model_name+".txt", "w+") as f:
        numround = int(p['numRounds'])
        f.write(str(bestMAE) + 'numRounds: '+str(numround))
        f.write(str(max_params))

In [56]:
kf_test = KFold(n_splits = 5, shuffle = True)#kfold를 사용하여 데이터셋 분리

In [58]:
for train_index, test_index in kf_test.split(arrays):# 1/5 로 데이터를 나눔. 원래 5번 교차검증을 해야하지만 여기선 생략.
    # split train/validation
    train_data, test_data  = arrays[train_index], arrays[test_index]
    train_labels, test_labels = labels[train_index], labels[test_index]
    print(len(train_index))

7090
7090
7090
7091
7091


In [46]:
len(train_data)

7091

In [47]:
len(test_data)

1772

In [59]:
ranges = {
    'numRounds': (1000, 2000),
    'eta': (0.03, 0.1),
    'gamma': (0, 10),
    'maxDepth': (4, 10),
    'minChildWeight': (0, 10),
    'subsample': (0, 1),
    'colSample': (0, 1),
}#학습에 따라 변경될 값들. 설정

In [38]:
optFunc = lambda numRounds, eta, gamma, maxDepth, minChildWeight, subsample, colSample: xgbCv(train, features, numRounds, eta, gamma, maxDepth, minChildWeight, subsample, colSample)

In [ ]:

# proxy through a lambda to be able to pass train and features
    
    bo = BayesianOptimization(optFunc, ranges)
    bo.maximize(init_points = 50, n_iter = 5, kappa = 2, acq = "ei", xi = 0.0)
    bestMAE = round((-1.0 * bo.res['max']['max_val']), 6)
    print("\n Best MAE found: %f" % bestMAE)
    print("\n Parameters: %s" % bo.res['max']['max_params'])#가장 최고의 파라미터 추출 나중에 리턴 받으면 될듯
    p = bo.res['max']['max_params']#가장 좋은 파라미터를 찾아서 p에 넣음.
    max_params = {
        'eta': float(p['eta']),
        'max_depth': int(p['maxDepth']),
        'subsample': max(min(p['subsample'], 1), 0),
        'objective': 'reg:linear',
        'silent': 1,
        'min_child_weight': int(p['minChildWeight']),
        'gamma': max(p['gamma'], 0),
        "objective": "reg:linear",
        "booster" : "gbtree",
        "eval_metric": "mae",
        "tree_method": 'auto',
        "silent": 1,
        "scale_pos_weight": 0.48
        }#가장 좋은 파라미터를 저장.